# Classifying Images as Birds or Airplanes


In [ ]:
**Dataset:** https://www.cs.toronto.edu/~kriz/cifar.html<br>**Similar tutorial:** https://livebook.manning.com/book/deep-learning-with-pytorch/chapter-7/v-13/166

## Import required libraries

PyTorch for building and training the neural network.

In [5]:
import torch

Torchvision

In [4]:
import torchvision

import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

Numpy for data manipulation and preprocessing

In [6]:
import matplotlib.pyplot as plt

## Data Visualisation (matplotlib)


## Data Pre-Processing (numpy) - Could be called 'creating x classification'